이번에는 축약 함수를 더 자세히 살펴본다. 순수한 함수적 관점에서 보면 축약은 재귀적으로 정의된다. 이러한 이유로 축약 아로길즘을 살펴볼 때는 먼저 재귀적인 알고리즘을 살펴본다.

일반적으로, 함수형 프로그래밍 언어의 컴파일러는 재귀함수를 최적화하여 함수의 꼬리재귀를 루프로 바꿔준다. 이렇게 하면 성능이 극적으로 향상된다. 파이썬의 관점에서 볼 때 순수한 재귀에는 항상 성능상의 한계가 있기 때문에 직접 손으로 꼬리재귀호출을 최적화해야 한다. 파이썬에서 사용할 쑤 있는 꼬리재쉬 최적화 기법은 명시적으로 for 루프를 사용하는 것이다.

sum(), count(), max(), min() 등의 축약 알고리즘을 살펴본다. 또 collections.Counter() 함수와 그 함수와 관련 잇는 groupby() 축약도 살펴본다. 그리고 구문 분석도, 토큰의 시퀀스를 더 복잡한 프로퍼티를 지니는 고차 컬렉션으로 바꿔주기 때문에 축약의 일부라는 사실에 대해 설명할 것이다.

### 수를 계산하는 간단한 재귀

재귀로 정의할 수 있는 모든 수치 연산을 고려할 수 있다. 더 자세한 내용이 궁금하다면, 수의 기본적인 성질에 대해 정의하는 페아노 공리계를 읽어보는 것이 좋은 출발점이 될 수 있다. 페아노 공리계를 살펴보면, 어떤 수 n의 다음 수 또는 어떤 수의 후계자는 S(n)이라는 더 원시적인 표현을 가지고 덧셈을 재귀적으로 정의할 수 있다는 사실을 알 수 있다.

설명을 단순화하기 위해 "전임자" 함수 P(n)을 정의할 수 있다고 가정해봏자. 전임자 함수는 n != 0인 경우, n=S(P(n))=P(S(n))이라는 성질을 만족한다.

두 자연수의 덧셈을 재귀적으로 정의할 수 있다.

S(n)이나 P(n) 대신 더 일반적으로 사용하는 n+1과 n-1을 사용단하면 add(a, b) = add(a-1, b+1)이라는ㄴ 것을 알 수 있다.

이름 다음과 같이 파이선으로 깔끔하게 옮길 수 있다.

In [1]:
def add(a,b):
    if a==0: return b
    else: return add(a-1, b+1)

단지 수학적인 표기를 파이썬으로 재배열했을 뿐이다. if를 맨 오른쪽으레 왼쪽으로 옮긴 것이 전부다.

일반적으로는 단순한 덧셈을 위해 파이썬으로 직접 함수를 만들지 않는다. 우리는 파이썬 구현이 여러 종류의 산술 연상에 대해 제공하는 기본 구현을 활용한다. 여기서 중요한 점은 모든 연산의 기초가 되는 산술 연산도 재귀적으로 정의할 수 있다는 것이다.

이러한 모든 재귀 정의에는 두 가지 경우가 들어 있다. 첫째는 함수의 결과 값을 직접적으로 정의하는 재귀적이지 않은 경우이고, 둘째는 함수의 값을 같은 함수에 다른 값을 적용해 얻은 결과를 사용해 계산해내는 재귀적인 경우다.

재귀가 끝난다는 것을 보장하려면, 재귀적인 경우에 사용하는 재귀호출이 어떻게 재귀적이지 않은 경우에 도닫ㄹ하도록 값을 계산하는지 살펴보는 것이 중요하다. 여기서 다루는 함수에는 따로 넣지 않았지만, 함수의 인자 값에 대한 제약이 있는 경우도 많다. 앞의 add() 함수에 assert a >= and b >= 0를 엏어 입력 값에 대한 제약을 표현할 수도 있다.

이러한 제약이 없다면, a-1이 재귀적이지 않은 a==0으로 접근한다는 것을 보장할 수 없다.

대부분의 경우 이러한 요소들은 명확하다. 하지만 이를 증명하기 어려운 경우가 가끔 있다. 한 가지 예를 시라쿠사 함수다. 이는 언제 종료하는지를 알기 어려운 함수의 전형적인 예다.

### 꼬리재귀호출 구현하기

일부 함수의 경우 재귀적 정의가 더 간결하고 함수를 잘 설명해주기도 한다. 가장 일반적인 예로는 factorial() 함수를 들 수 있다.

In [2]:
def fact(n):
    if n==0: return 1
    else:
         return n * fact(n-1)

이 정의의 장점은 단순하다는 것이다. 하지만 파이썬의 재귀 깊이 제한으로 인해 fact(997)보다 큰 수를 계산할 수 없다. 게다가 1000!sms 2,568 자리의 수이기 때문에 일반적인 부동 소수점 범위를 벗어난다. 부동 소수점 수는 보통 10^300 정도의 범위다. 실용적으로 볼 때 이를 log gamma 함수로 바꾸는 것이 더 일반적이다. 이 경우 큰 부동 소수점 값에 대해서도 잘 작동할 것이다.

이 함수는 전형적인 꼬리재귀함수다. 함수가 마지막으로 수행하는 식은 함수 자신에 새로운 값을 지정하여 호출하는 것 뿐이다. 최적화 컴파일러는 함수 호출 스택 관리를 실행 속도가 매우 빠른 루프로 바꿀 수 있다.

파이썬이 그러한 최적화를 제공하지 않기 때문에 꼬리재귀를 보고 이를 최적화할 방향을 알아볼 수 있는 눈을 가질 필요가 있다. 여기서는 함수의 인자가 n에서 n-1로 점진적으로 변하는 과정이 계속된다. 이는 우리가 수의 시퀀스를 만들어 내고, 각 값의 곲을 계산하는 축약을 수행함을 의미한다.

순수 함수에서 한 걸음 밖으로 나오면, 명령형 facti()를 다음과 같이 정의할 수 있다.

In [3]:
def facti(n):
    if n == 0: return 1
    f =1
    for i in range(2, n):
        f = f*i
    return f

이 계승 함수는 1000!을 넘어서는 값도 계산할 수 있다. 이는 순수한 함수형 함수가 아니다. 꼬리재귀를 i변수의 값에 의존하면서 계산 상태를 유지하는 루프로 최적화했다.

일반적으로 파이썬이 자동으로 꼬리재귀 최적화를 해주지 않기 때문에 파이썬 대신 이러한 최적화를 수행해야 한다. 하지만 이러한 종류의 최적화가 실제로 별 도움이 되지 않는 경우도 있다. 그러한 경우를 몇 가지 살펴보자.

### 재귀를 그대로 남겨두기

경우에 따라, 재귀적 정의가 실제로도 최적일 수 있다. 일부 재귀 작업의 복잡도를 $O(n)$에서 $O(log_2 n)$으로 바꿔주는 분할 정복 전략을 따른다. 이러한 경우의 한 예는 임의의 제곱수 계산을 2의 거듭제곱을 사용해 계산하는 알고리즘이다. 

$a^n = 1$ n = 0인 경우

$a^n = a * a^{n-1}$ n이 홀수인 경우

$a^n = (a^{n/2})^2$ n이 짝수인 경우

계산 가정을 세 가지로 나눴다. 파이썬에서도 재귀를 사용해 이를 쉽게 구현할 수 있다. 다음 코드를 살펴보자.

In [4]:
def fastexp(a, n):
    if n == 0: return 1
    elif n % 2 ==1: return a*fastexp(a, n-1)
    else:
        t = fastexp(a, n//2)
        return t*t

이 함수에는 세 가지 경우가 들어 있다. 기본적인 경우 fastext(a, 0) 메서드는 1로 정의된다. 다른 두 경우는 서로 다른 접근 방법을 선택한다. 홀수의 경우 fastexp() 메서드는 재귀적으로 지수 n을 1 감소시켜 자신을 호출한다. 이 경우에는 간단한 꼬리재귀로 처리할 수 있다.

하지만 짝우의 경우 fastexp()는 n/2를 사용해 문제를 원래 크기의 절반으로 줄여버린다. 문제 크기가 2배 줄었기 때문에 이 경우 처리 속도가 엄청나게 빨라진다.

이러한 함수는 수비게 꼬리재귀 루프로 바꿀 수 없다. 이미 이 알고리즘이 최적이기 때문에 실제 이를 더 최적화할 필요도 없다. 파이썬의 재귀 깊이 제한인 n <= 2^1000도 상당히 큰 범위의 계산을 허용한다고 볼 수 있다.

### 처리하기 어려운 꼬리재귀 다루기

피보나치 수는 단순한 꼬리재귀로 최적화할 수는 없다. 하지만 이 함수를 꼬리재귀로 최적화하지 않으면, 너무 느려서 쓸모가 없다. 다음은 정의를 직접 따라한 구현이다.

In [6]:
def fib(n):
    if n == 0: return 0
    if n == 1: return 1
    return fib(n-1) + fib(n-2)

이 코드를 실행하면 다중 재귀의 문제를 볼 수 있다. 

파이썬이 식을 왼쪽에서 오른쪽으로 계산하기 때문에 fib(1000)까지의 값을 계산할 수 있다. 

다음은 단순한 재귀 대신 상태가 있는 변수를 사용해 전체 알고리즘을 재작성한 것이다.

In [7]:
def fibi(n):
    if n == 0: return 0
    if n == 1: return 1
    f_n2, f_n1 = 1, 1
    for i in range(3, n+1):
        f_n2, f_n1 = f_n1, f_n2+f_n1
    return f_n1

여기서 중요한 것을 뻔하게 재작성하는 방벙으로는 재귀를 쉽게 최적화할 수는 없다는 사실이다. 재귀를 명령형 버전으로 대치하려면 알고맂믕ㄹ 자세히 살펴보고 얼마나 많은 상태 정보를 유지해야 할 것인지 결정해야 한다.

### 재귀를 사용해 컬렉션 처리하기

컬렉션을 다루는 경우에도 처리를 재귀적으로 정의할 수 있다. 예를 들어, map() 함수를 재귀로 정의할 수 있다.

다음은 map() 함수를 순수하게 함수적인 재귀 버전으로 작성한 것이다.

In [8]:
def mapr(f, collection):
    if len(collection) == 0: return []
    return mapr(f, collection[:-1]) + [f(collection[-1])]

mapr(f, []) 메서드의 값은 빈 list 객체로 정의된다. mapr() 함수에 원소가 있는 리스트를 적용한 값은 함수 f를 list1의 마지막 원소에 대해 호출한 결과를 mapr()을 리스트의 앞부분에 호출해 얻은 리스트 뒤에 붙인 것이다.

mapr() 함수도 파이썬의 옛 map() 함수처럼 list 객체를 만들어 낸다. 파이썬 3의 map() 함수는 반복 가능 객체를 반환하며, 이는 꼬리재귀 최적화의 좋은 예가 아니다.

비록 이러한 정의가 우아하기는 하지만, 꼬리재귀를 최적화하지 않았다는 단점이 있다. 꼬리재귀를 최적화하면 1,000개 이상의 원소를 처리할 수 있고, 단순한 재귀 버전보다 실행 성능도 훨씬 좋아진다.

### 컬렉션에 대한 꼬리 호출 최적화

컬렉션을 처리하는 일반적인 방법에는 두 가지가 있다. 첫째는 제너레이터 식을 반환하는 고차 함수를 사용하는 것이고, 둘째는 각 원소를 처리하는 for 루프를 사용하는 함수를 만드는 것이다. 이 두 가지 필수적인 패턴은 서로 비슷하다.

다음은 내장 map() 함수처럼 작동하는 고차 함수다.

In [9]:
def mapf(f, C):
    return (f(x) for x in C)

원하는 매핑을 만들어 내는 제너레이터 식을 반환했다. 명시적으로 for 루프를 사용해 꼬리재귀를 최적화했다.

다음은 같은 결과를 내놓는 제너레이터 함수다.

In [10]:
def mapg(f, C):
    for x in C:
        yield f(x)

이 코드는 최적화를 위해 완전한 for문을 사용했다.

두 경우 모두 결과는 반복 가능 객체다. 다음과 같이 해야 이를 시퀀스 객체로 실체화할 수 있다.

In [11]:
list(mapg(lambda x: 2**x, [0, 1, 2, 3, 4]))

[1, 2, 4, 8, 16]

성능이나 규모 확장성을 생각보자. 이러한 종류의 꼬리재귀 최적화는 파이썬 프로그램에서 필수적이다. 하지만 코드를 덜 순수한 함수형 프로그램으로 만든다. 또한 최적화로 얻는 이점이 순수성을 잃는 대가보다 더욱 크다. 간결하면서 알아보기 쉬운 함수형 설계의 이점을 살리기 위해서는 이러한 방식으로 덜 순수한 함수들이 마치 완전한 재귀함수인 것처럼 다루는 것이 도움이 될 수 있다.

실용적인 관점에서 이것이 의미하는 바는 컬렉션을 처리하는 함수에 추가로 상태의 처리를 넣는 것을피해야 한다는 것이다. 함수형 프로그래밍의 핵심 교리는 마든 프로그램의 일부분이 함수적으로 덜 순수하다 하더라도 여전히 유효하다는 것이다. 

### 축약과 겹치기 - 많은 값을 한 가지 값으로 줄이기

파이썬에서는 prod 함수를 다음과 같이 재귀적으로 정의할 수 있다.

In [13]:
def prodrc(collection):
    if len(collection) == 0: return 1
    return collection[0] * prodrc(collection[1:])

기술적으로는 이 정의가 맞다. 수학적 표기를 파이썬으로 다씨 쓰는 것은 쉽다. 하지만 중간에 너무 많은 리스트 객체를 만들어 내기 때문에 이 함수는 최적과는 거리가 멀다. 또 명시적인 컬렉션만 처리할 수 있다는 단점도 있다. 이 함수를 반복 가능 객체에 직접 사용할 수는 없다. 이를 약간 바꿔 반복 가능 객체에 적용할 수 있게 만들면, 중간중간 생기는 컬렉션 객체를 없앨 수 있다. 다음은 반복 가능 객체를 사용하면서 재귀적으로 곱셈을 계산하는 함수다.

In [14]:
def prodri(iterable):
    try:
        head=next(iterable)
    except StopIteration:
        return 1
    return head*prodri(iterable)

반복 가능 객체의 원소 개수를 알아내기 위해 len()을 호출할 수는 없다. 우리가 할 수 있는 것은 반복 가능 객체의 다음 원소를 가져오는 것뿐이다. 그 객체에 원소가 없다면 다음 숸소를 가져오려는 시도는 StopIteration 예외를 발생시킨다. 원소가 있다면 그 원소와 반복 가능 객체의 나머지 원소의 곱을 서로 곱한다. 이 함수를 시험해보기 위해서는 반드시 iter() 함수를 사용해 실체화된 시퀀스 객체로부터 반복 가능 객체를 만들어야 한다. 사용 가능한 반복 가능 객체가 있는 경우라면 그냥 그 객체를 쓰면 된다. 다음 예를 살펴보자.

In [15]:
prodri(iter([1, 2, 3, 4, 5, 6, 7]))

5040

여기에 있는 재귀적 정의는 눈에 보이는 상태나 파이썬의 다른 명령형 기능에 의존하지 않는다. 이 함수가 좀 더 순수 함수이기는 하지만, 이 역시 원소가 1,000개 이하인 경우에만 처리할 수 있다. 실용성을 위해 다음과 같이 명령형의 축약 함수를 사용할 수 있다.

In [16]:
def prodri(iterable):
    p = 1
    for n in iterable:
        p *= n
    return p

이렇게 하면 재귀 깊이의 제한에 영향을 받지 않는다. 이 함수에는 필요한 꼬리재귀 최적화가 들어 있다. 더 나아가 대상 객체가 시퀀스냐, 반복 가능 객체이냐와 관계 없이 잘 작동할 것이다.

다른 함수형 언어에서는 이러한 함수는 'foldl 연산'이라 부른다. 연산자가 반복 가능 컬렉션의 값을 왼쪽에서 오른쪽으로 겹쳐 나간다. 이는 계산을 오른쪽에서 왼쪽으로 진행해 나가는, foldr이라 불리는 재귀 정의와는 차이가 있다.

최적화 컴파일러와 지연 계산이 있는 언어에서 오른쪽 접기와 왼쪽 접기는 중간 결과가 만들어지는 순서를 결정한다. 경우에 따라 둘 사이에 심각한 성능상 창이가 나타날 수도 있지만, 그러한 차이가 항상 분명한 것은 아니다. 

### Counter로 매핑 만들기

collection.Counter와 같은 매핑을 사용하면 컬렉셕에 있는 원소들을 특정 값에 의해 그룹화하여 개수를 계산하는 축약을 수행할 때 상당한 최적화를 달성할 수 있다. 데이터를 그룹화할 때 더 함수적인 접근 방법은 원래의 컬렉션을 정렬하고, 재귀적 루프를 사용하여 각 그룹의 시작 부분을 식별하는 것이다. 이렇게 하려면 원 데이터를 실체화해야 하고, 정렬에 $O(n log_n)$이 들고, 그 후 각 키에 따른 합계나 개수를 계산하기 위한 축약에도 비용이 든다.

다음 제너레이터를 사용해 그룹을 나눌 기준 거리의 시퀀스를 만들 수 있다.

In [21]:
trip = (((37.5, -75.5), (37.6, 12), 17.7), ((37.5, -75.5), (37.6, 12), 17.7), ((37.5, -75.5), (37.6, 12), 17.7))
qauntized = (5*(dist//5) for start, stop, dist in trip)

다음 식은 거리와 빈도를 연결하는 매핑을 만든다.

In [22]:
from collections import Counter
Counter(qauntized)

Counter({15.0: 3})

이 객체에는 상태가 있다. 이 객체는 기술적으로는 명령형 객체지향 프로그래밍에 의해 만들어진 것이다. 하지만 이 객체를 함수처럼 사용할 수 있기 때문에 함수형 프로그래밍의 아이디어를 살린 설계에도 잘 들어맞는다.

### 정렬을 사용해 매핑 만들기

Counter 클래스가 없이 같은 문제의 해법을 구현하고 싶다면, 정렬한 후 그룹을 만드는 더 함수적인 방식을 사용할 수 있다. 다음은 일반적인 알고리즘을 보여준다.

In [23]:
def group_sort(trip):
    def group(data):
        previous, count = None, 0
        for d in sorted(data):
            if d == previous:
                count += 1
            elif previous is not None:
                yield previous, count
                previous, count = d, 1
            elif previous is None:
                previous, count = d, 1
            else:
                raise Exception("Bad has design problem.")
        yield previous, count
    quantized = (5*(dist//5) for start, stop, dist in trip)
    return dict(group(quantized))

내부의 group() 함수는 정렬한 데이터에 대해 루프를 수행한다. 어떤 원소를 이미 살펴봤다면 count 값을 1 증가시킨다. 어떤 원소가 앞에서 살벼폰 previous와 다르고, previous가 None도 아니라면, 그룹이 바뀐 것이므로 previous 값과 빈도수 count 값을 내보내고, 새로운 count 값을 누적하기 시작한다. 세 번째 조건은 오직 한 번만 일어난다. privious가 설정되지 않은 경우라면 현재 처리 중인 값이 최초의 값이기 때문에 그 정보를 저장해야 한다.

함수의 마지막 줄은 그룹화한 원소들로부터 딕셔너리를 만든다. 이 딕셔너리는 Counter의 딕셔너리와 비슷하다. 가장 큰 차이는 Counter() 함수에는 기본 딕셔너리가 제공하지 않는 most_commin()이라는 메서드 함수가 있다는 점이다.

elif previous is None이라는 조건은 조금 눈에 거슬리며, 약간의 부가 비용도 발생한다. 이 elif절을 없애는 것은 그리 어렵지 않다.

이 elif절을 없애기 위해서는 내부 group() 함수를 초기화할 때 약간의 처리를 추가해야 한다.

In [24]:
def group(data):
    sorted_data = iter(sorted(data))
    previous, count = next(sorted_data), 1
    for d in sorted_data:
        if d == previous:
            count += 1 
        elif previous is not None:
            yield previous, count
            previous, count = d, 1
        else:
            raise Exception("Bad bad design problem.")
    yield previous, count

여기서는 previous 값을 초기화하기 위해 정렬한 데이터의 첫 번째 원소를 사용한다. 그 후 나머지 원소를 루프 안에서 처리한다. 이 설계는 재귀를 설계할 때 최초 원소를 사용해 재귀를 초기화하고, 각 재귀호출에서는 다음 원소를 가져오거나 처리에 필요한 원소가 더 이상 없는 경우 None을 사용하는 것과 비슷하다.

여기서 본 작업에 itertools.groupby()를 사용할 수도 있다.

### 키 값에 따라 데이터를 그룹화하거나 분할하기

그룹화한 데이터에 적용할 수 있는 축약의 종류는 무한하다. 여러 독립 변수와 종속 변수로 이뤄진 데이터가 있다면, 그 데이터를 독립 변수별로 분할하고, 각 분할에 속한 값들의 최대, 최소, 평균, 표준편차 등을 계산할 수 있다.

좀 더 복잡한 축약을 수행하기 위해 필수적인 것은 모든 데이터 값을 각 그룹에 넣는 기능이다. Counter() 함수는 단순히 동일한 원소의 개수를 셀 뿐이다. 키 값에 따라 원래 객체를 열거한 시퀀스를 만들고 싶다.

각 5마일 단위의 그룹에는 단순히 개수만 들어 있는 것이 아니라, 해당 거리 그룹에 속하는 모든 구간의 컬렉션이 들어 있다. 이러한 분할을 재귀로 생각할 수도 있고, defaultdict(list) 객체를 응용한 것으로 생각할 수도 있다. 상대적으로 좀 더 쉽기 때문에 groupby() 함수의 재귀적 정의를 먼저 살펴본다. 

groupby(C, key) 메서드를 빈 컬렉션 C에 적용하면 빈 딕셔너리 dict()가 나온다. 빈 defalutdict(list) 객체가 나오면 좀 더 유용할 것이다.

비어 있지 않은 컬렉션의 경우, 머리 원소인 C[0]을 처리하고, 재귀적으로 꼬리인 C[1:]을 처리해야 한다. 컬렉션을 이렇게 나누기 위해 head, *tail = C를 사용할 수 있다. 다음을 살펴보자.

In [25]:
C = [1, 2, 3, 4, 5]

In [26]:
head, *tail = C

In [27]:
head

1

In [28]:
tail

[2, 3, 4, 5]

In [29]:
def group_by(key, data):
    def group_into(key, collection, dictionary):
        if len(collection) == 0:
            return dictionary
        head, *tail = collection
        dictionary[key(head)].append(head)
        return group_into(key, tail, dictionary)
    return group_into(key, data, defaultdict(list))

내부 함수는 앞에서 설명한 재귀를 정의한다. 빈 컬렉션의 경우 제공 받은 딕셔너리를 반환한다. 비어 있지 않은 컬렉션의 경우 머리와 꼬리를 나눈다. 머리를 사용해 딕셔너리를 갱신하고, 꼬리를 재귀적으로 사용해 나머지 모든 원소에 대해 딕셔너리를 갱신한다.

파이썬의 기본 값 기능을 사용해도 쉽게 전체를 한 함수 정의에 넝을 수는 없다. 즉, 다음과 같은 코드를 사용할 수 없다.

In [31]:
def group_by(key, data, dictionary=defauldict(list)):

이렇게 한다면 group_by 함수를 사용하는 모든 호출에서 동일한 defaultdict(list) 객체를 공유할 것이다. 파이썬은 기본 값을 한 번만 만든다. 변경 가능한 객체를 기본 값으로 쓰는 경우는 드물다. 변경 불간으한 기본 값을 처리하기 위해 복잡한 의사 결저을 코드에 넣는 것보다 함수 정의를 내포시키는 것이 낫다. 내포 함수를 감싸고 있는 함수는 내포된 함수에게 적절한 인자를 초기화해 제공한다.

꼬리재귀호출 최적화를 수행할 때 명령형 버전의 코드 중 핵심적인 부분은 재귀적인 정의에 부합해야 한다. 여기서 재작성한 것이 원래 함수와 같게 만들기 위한 것임을 보여주기 위해 코드에서 해당되는 부분을 강조했다. 

### 더 일반적인 그룹화 축약 작성하기

원 데이터를 분할하고 나면, 각 분할에 속한 컬렉션에 대해 여러 가지 축약을 수행할 수 있다. 

튜플을 분해하기 위한 두우미 함수는 다음과 같이 정의할 수 있다.

In [33]:
start = lambda s, e, d :s
end = lambda s, e, d :e
dist = lambda s, e, d :d
latitude = lambda lat, lon: lat
longitude = lambda lat, lon: lon

이러한 도우미 함수의 인자로는 튜플에 * 연산자를 사용하여 튜플의 각 원소를 람다의 인자로 전달애햐 한다. 튜플을 s, e, p 매개변수로 분해하고 나면 이름에 따라 적절한 인자를 반환하는 것은 간단한 일이다. 이렇게 하는 것이 tuple_arg[2]와 같은 메서드를 사용하는 것보다 훨씬 더 깔끔하다.

### 파일 구문 분석기 작성하기

종종 파일 구문 분석을 축약의 일종으로 생각할 수도 있다. 여러 언어에는 그 언어를 이루는 하위 수쿤의 토큰과 그러한 토큰이 이루는 상위 수준의 구조와 있다. XML 파일의 태그, 태그 이름, 애트리뷰터 이름 등은 하위 수준의 문법을 구성하며, XML이 표현하는 구조는 상위 수준의 문법을 이룬다.

하위 수준의 어휘 스캐닝을 개별 문자를 받아 토큰으로 그룹화해주는 축약이라 할 수 있다. 이 구조는 파이썬의 제너레이터 함수 디자인 패턴에 잘 들어맞는다. 가끔은 다음과 같은 함수를 작성할 수 있다.

In [38]:
def lexical_scan(some_source):
    for char in some_source:
        if some_pattern completed: yield token
        else: accumulate token